In [1]:
!pip install torch

In [ ]:
# !wget 'https://app.roboflow.com/ds/2C6eKSmGsh?key=pxrkodvP4R'
!unzip '/content/drive/MyDrive/grid finale/full_final_dataset_man_split_manip_even.zip' -d '/content/datasets'

TRY NEW


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import torchvision.models as models

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load a pre-trained ResNet-50 model with random initialization
# model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(device)
model = models.mobilenet_v2(pretrained=True).to(device)
model.classifier[1] = nn.Linear(1280, 3).to(device)
# # model.apply(initialize_weights)
for param in model.parameters():
    param.requires_grad = False


# # Then, unfreeze the last 10 layers
for param in list(model.parameters())[-5:]:
    param.requires_grad = True
# for param in model.parameters():
#     param.requires_grad = True

# Initialize the model weights randomly
# def initialize_weights(module):
#     if isinstance(module, nn.Conv2d):
#         nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')
#         if module.bias is not None:
#             nn.init.constant_(module.bias, 0)
#     elif isinstance(module, nn.BatchNorm2d):
#         nn.init.constant_(module.weight, 1)
#         nn.init.constant_(module.bias, 0)

# model.apply(initialize_weights)

# Define a custom head for the ResNet-50 model to adapt it to the MNIST dataset
# num_classes = 3  # MNIST has 10 classes
# model.fc = nn.Linear(512, num_classes).to(device)  # Change the fully connected layer and move to GPU

# Data loading and preprocessing
transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    # transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    # transforms.Normalize([0.6319697,  0.53482469, 0.49866286],[0.36895301, 0.30741617, 0.17927112]),
])

In [15]:
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [22]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)

+----------------------+------------+
|       Modules        | Parameters |
+----------------------+------------+
| features.18.0.weight |   409600   |
| features.18.1.weight |    1280    |
|  features.18.1.bias  |    1280    |
| classifier.1.weight  |    3840    |
|  classifier.1.bias   |     3      |
+----------------------+------------+
Total Trainable Params: 416003


416003

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import transforms, datasets
# from torch.utils.data import DataLoader, random_split
# import torchvision.models as models
import random
import os

# # Check if GPU is available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# # Load a pre-trained ResNet-50 model with random initialization
# model = models.resnet50(pretrained=False).to(device)

# # Initialize the model weights randomly
# def initialize_weights(module):
#     if isinstance(module, nn.Conv2d):
#         nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')
#         if module.bias is not None:
#             nn.init.constant_(module.bias, 0)
#     elif isinstance(module, nn.BatchNorm2d):
#         nn.init.constant_(module.weight, 1)
#         nn.init.constant_(module.bias, 0)

# model.apply(initialize_weights)

# # Define a custom head for the ResNet-50 model to adapt it to the MNIST dataset
# num_classes = 3  # MNIST has 10 classes
# model.fc = nn.Linear(2048, num_classes).to(device)  # Change the fully connected layer and move to GPU

# # Data loading and preprocessing
# transform = transforms.Compose([
#     # transforms.Resize((224, 224)),
#     transforms.Grayscale(num_output_channels=3),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# # Load MNIST dataset with a smaller training dataset
# full_dataset = datasets.MNIST(
#     root='./data',
#     train=True,
#     transform=transform,
#     download=True
# )

batch_size =16
dataset_root = '/content/datasets'  # Replace with the actual path


# <----------------------------------------------------->


# Define the data transformation you want to apply to the images

# Define a custom function to create a dataset with 20% of "0" class and all of "1" and "2" classes
# def create_custom_dataset(root, transform, class_to_select):
#     custom_samples = []

#     for class_name in os.listdir(root):
#         class_folder = os.path.join(root, class_name)

#         # Select 20% of "0" class, and all of "1" and "2" classes
#         if class_name == class_to_select:
#             all_images = [os.path.join(class_folder, filename) for filename in os.listdir(class_folder)]
#             num_images_to_select = int(0.8 * len(all_images))
#             selected_images = random.sample(all_images, num_images_to_select)
#         else:
#             selected_images = [os.path.join(class_folder, filename) for filename in os.listdir(class_folder)]

#         class_label = int(class_name)  # Assuming class folders are named as integers

#         for image_path in selected_images:
#             custom_samples.append((image_path, class_label))

#     custom_dataset = datasets.ImageFolder(root=root, transform=transform)
#     custom_dataset.samples = custom_samples

#     return custom_dataset

# # Create custom datasets and data loaders for train, valid, and test
# train_dataset = create_custom_dataset(os.path.join(dataset_root, 'train'), transform, class_to_select='0')
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# val_dataset = create_custom_dataset(os.path.join(dataset_root, 'valid'), transform, class_to_select='0')
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# test_dataset = create_custom_dataset(os.path.join(dataset_root, 'test'), transform, class_to_select='0')
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# # <------------------------------------------------------>
# zero_folder_path = '/content/datasets/train/0'

# Get the list of all image file paths in the "zero" folder
# all_zero_images = [os.path.join(zero_folder_path, filename) for filename in os.listdir(zero_folder_path)]

# Calculate the number of images to select (20% of the total)
# num_images_to_select = int(0.2 * len(all_zero_images))

# Randomly select 20% of the images from the "zero" folder
# selected_zero_images = random.sample(all_zero_images, num_images_to_select)

# Create a custom dataset from the selected images
# custom_train_dataset = datasets.ImageFolder(root=zero_folder_path, transform=transform)
# custom_train_dataset.samples = [(image_path, 0) for image_path in selected_zero_images]  # Assuming "zero" class is labeled as 0

# Create a DataLoader for the custom training dataset
# custom_train_loader = DataLoader(custom_train_dataset, batch_size=batch_size, shuffle=True)

# Create data loaders for train, validation, and test sets
train_dataset = datasets.ImageFolder(root=f'{dataset_root}/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = datasets.ImageFolder(root=f'{dataset_root}/valid', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = datasets.ImageFolder(root=f'{dataset_root}/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training parameters
learning_rate = 0.001
num_epochs = 6

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

# Training loop with validation
best_val_accuracy = 0.0
for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Print training statistics
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {total_loss / len(train_loader)}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

    # Save the model with the best validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model.pth')

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Testing and evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [27]:
!mv "/content/final_dataset_manual_manip_even.pth" "/content/drive/MyDrive/grid finale"

mv: cannot stat '/content/final_dataset_manual_manip_even.pth': No such file or directory


In [ ]:
!pip install qiskit

In [ ]:
!pip install qiskit-aer

In [ ]:
from PIL import Image
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_bloch_multivector

# Load your grayscale image and convert it to a numpy array
# Example: Replace 'image_path' with the path to your image
image = Image.open('/content/morphle_lab_q1_ss.png')
image = image.convert('L')  # Convert to grayscale
image_array = np.array(image)

# Sample grayscale image (replace with your image data)
# image_array = np.array([[128, 64, 192],
#                         [64, 128, 64],
#                         [192, 64, 128]])

# Create a quantum circuit with enough qubits to represent pixel values
num_qubits = len(image_array.flatten())
circuit = QuantumCircuit(num_qubits, num_qubits)

# Encode pixel values into quantum states
for i, pixel_value in enumerate(image_array.flatten()):
    # Normalize pixel value to [0, 1]
    normalized_value = pixel_value / 255.0

    # Encode the pixel value as a quantum state (amplitude)
    circuit.ry(2 * np.arcsin(np.sqrt(normalized_value)), i)

# Simulate the quantum circuit to get the quantum state
simulator = Aer.get_backend('qasm_simulator')
job = assemble(transpile(circuit, simulator))
result = simulator.run(job).result()
quantum_state = result.get_statevector()

# Visualize the quantum state (for demonstration)
plot_bloch_multivector(quantum_state)


In [ ]:
+